With conversational agents we can split a process and define various agents for each task in the process and then ask to each agent to do separate task and finally we get the final results.
See this example: we split the task into subtasks and defined amy agents and then each agent has done them!

In [1]:
!pip install pyautogen


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import os
openAI_API_Key=os.environ['openAI_API_Key']
llm_config = {"model": "gpt-3.5-turbo"}

In [3]:
import autogen

C:\Users\moham\anaconda3\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,
C:\Users\moham\anaconda3\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
from autogen import ConversableAgent

In [5]:
onboarding_personal_information_agent = ConversableAgent(
    name="Onboarding Personal Information Agent",
    system_message='''You are a helpful customer onboarding agent,
    you are here to help new customers get started with our product.
    Your job is to gather customer's name and location.
    Do not ask for other information. Return 'TERMINATE' 
    when you have gathered all the information.''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
)

In [6]:
onboarding_topic_preference_agent = ConversableAgent(
    name="Onboarding Topic preference Agent",
    system_message='''You are a helpful customer onboarding agent,
    you are here to help new customers get started with our product.
    Your job is to gather customer's preferences on news topics.
    Do not ask for other information.
    Return 'TERMINATE' when you have gathered all the information.''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
)

In [7]:
customer_engagement_agent = ConversableAgent(
    name="Customer Engagement Agent",
    system_message='''You are a helpful customer service agent
    here to provide fun for the customer based on the user's
    personal information and topic preferences.
    This could include fun facts, jokes, or interesting stories.
    Make sure to make it engaging and fun!
    Return 'TERMINATE' when you are done.''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "terminate" in msg.get("content").lower(),
)

In [8]:
customer_proxy_agent = ConversableAgent(
    name="customer_proxy_agent",
    llm_config=False,
    code_execution_config=False,
    human_input_mode="ALWAYS",
    is_termination_msg=lambda msg: "terminate" in msg.get("content").lower(),
)

Now, you can craft a series of tasks to facilitate the onboarding process.

In [9]:
chats = [
    {
        "sender": onboarding_personal_information_agent,
        "recipient": customer_proxy_agent,
        "message": 
            "Hello, I'm here to help you get started with our product."
            "Could you tell me your name and location?",
        "summary_method": "reflection_with_llm",
        "summary_args": {
            "summary_prompt" : "Return the customer information "
                             "into as JSON object only: "
                             "{'name': '', 'location': ''}",
        },
        "max_turns": 2,
        "clear_history" : True
    },
    {
        "sender": onboarding_topic_preference_agent,
        "recipient": customer_proxy_agent,
        "message": 
                "Great! Could you tell me what topics you are "
                "interested in reading about?",
        "summary_method": "reflection_with_llm",
        "max_turns": 1,
        "clear_history" : False
    },
    {
        "sender": customer_proxy_agent,
        "recipient": customer_engagement_agent,
        "message": "Let's find something fun to read.",
        "max_turns": 1,
        "summary_method": "reflection_with_llm",
    },
]

In [10]:
from autogen import initiate_chats

chat_results = initiate_chats(chats)


********************************************************************************
Starting a new chat....

********************************************************************************
Onboarding Personal Information Agent (to customer_proxy_agent):

Hello, I'm here to help you get started with our product.Could you tell me your name and location?

--------------------------------------------------------------------------------


C:\Users\moham\anaconda3\lib\site-packages\autogen\agentchat\chat.py:47: UserWarning: Repetitive recipients detected: The chat history will be cleared by default if a recipient appears more than once. To retain the chat history, please set 'clear_history=False' in the configuration of the repeating agent.
  warnings.warn(


Replying as customer_proxy_agent. Provide feedback to Onboarding Personal Information Agent. Press enter to skip and use auto-reply, or type 'exit' to end the conversation: Ali from Montreal
customer_proxy_agent (to Onboarding Personal Information Agent):

Ali from Montreal

--------------------------------------------------------------------------------
Onboarding Personal Information Agent (to customer_proxy_agent):

Thank you, Ali from Montreal. Is there anything else you need assistance with?

--------------------------------------------------------------------------------
Replying as customer_proxy_agent. Provide feedback to Onboarding Personal Information Agent. Press enter to skip and use auto-reply, or type 'exit' to end the conversation: no
customer_proxy_agent (to Onboarding Personal Information Agent):

no

--------------------------------------------------------------------------------

********************************************************************************
Startin

**Print out the summary**

In [11]:
for chat_result in chat_results:
    print(chat_result.summary)
    print("\n")

{'name': 'Ali', 'location': 'Montreal'}


Ali from Montreal is interested in reading about Generative AI.


Ali from Montreal is interested in reading about Generative AI, which involves machines not only understanding data but also creating new data based on what they've learned. Enjoy exploring the world of Generative AI!




**Cost Calculation**

In [12]:
for chat_result in chat_results:
    print(chat_result.cost)
    print("\n")

{'usage_including_cached_inference': {'total_cost': 0.000136, 'gpt-3.5-turbo-0125': {'cost': 0.000136, 'prompt_tokens': 185, 'completion_tokens': 29, 'total_tokens': 214}}, 'usage_excluding_cached_inference': {'total_cost': 0.000136, 'gpt-3.5-turbo-0125': {'cost': 0.000136, 'prompt_tokens': 185, 'completion_tokens': 29, 'total_tokens': 214}}}


{'usage_including_cached_inference': {'total_cost': 5.1000000000000006e-05, 'gpt-3.5-turbo-0125': {'cost': 5.1000000000000006e-05, 'prompt_tokens': 66, 'completion_tokens': 12, 'total_tokens': 78}}, 'usage_excluding_cached_inference': {'total_cost': 5.1000000000000006e-05, 'gpt-3.5-turbo-0125': {'cost': 5.1000000000000006e-05, 'prompt_tokens': 66, 'completion_tokens': 12, 'total_tokens': 78}}}


{'usage_including_cached_inference': {'total_cost': 0.000339, 'gpt-3.5-turbo-0125': {'cost': 0.000339, 'prompt_tokens': 273, 'completion_tokens': 135, 'total_tokens': 408}}, 'usage_excluding_cached_inference': {'total_cost': 0.000339, 'gpt-3.5-turbo-0125